### Dataprep

#### Ler o arquivo fonte

In [99]:
import pandas as pd

## ler o csv bruto
df = pd.read_csv('./salarios_recife_outubro_2013.csv', sep=',', header=0)

df

,ORGAO,NOME,CATEGORIA,CARGO,FUNCAO,R$SALARIO
0,SECRETARIA DE EDUCACAO,9/16/13,ESTATUTARIO,TECNICO EDUCACIONAL,NaN,R$1902.24
1,SECRETARIA DE EDUCACAO,GOV. PERNAMBUCO **** CONSISTHR **...,ESTATUTARIO,ASSIST ADMINIST EDUCACIONAL,NaN,R$2667.97
2,SECRETARIA DE EDUCACAO,ARNEY SPINELI GUIMARAES,ESTATUTARIO,TECNICO EDUCACIONAL,NaN,R$2302.24
3,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZACAO,ARNEY SPINELI GUIMARAES,CTD - CONTRATO TEMPO DETERMINADO,PSICOLOGO - CTD,NaN,R$1770
4,SECRETARIA DE EDUCACAO,ITA DE CASSIA FISCHER DEL PINO,ESTATUTARIO,PROFESSOR,NaN,R$2325.26
...,...,...,...,...,...,...
133907,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,SIMONE VALERIA DE GUSMAO SANTOS,CTD - CONTRATO TEMPO DETERMINADO,ASSISTENTE ADMINISTRATIVO CTD,NaN,R$1100
133908,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,THAIS SOARES FONSECA,COMISSIONADO,ASSESS SUP AO SEC-ASS NAT TEC,ASSESS SUP AO SEC-ASS NAT TEC,R$4429.61
133909,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,VIVIANE DA SILVA LUCENA PATRIOTA,CTD - CONTRATO TEMPO DETERMINADO,TECNICO NA AREA SOCIAL - CTD,NaN,R$2200
133910,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,WAGNER LEITE DE OLIVEIRA,CTD - CONTRATO TEMPO DETERMINADO,TECNICO AGRICOLA - CTD,NaN,R$1635


#### Lidar com nulos

In [100]:
df.apply(lambda x : sum(x.isnull()), axis=0)

ORGAO            0
NOME             0
CATEGORIA        0
CARGO            0
FUNCAO       91556
R$SALARIO        0
dtype: int64

A coluna de "FUNCAO" possui uma quantidade enorme de nulos (68% das linhas), podendo enviezar o modelo para favorecer salários maiores para funções nulas. Visto que apesar da função de um cargo ser um fator importante para seu salário, não há uma maneira satisfatória de subsistituir os dados nulos portanto decidi remover a coluna completamente

In [101]:
df = df.drop('FUNCAO', axis=1)

#### Arrumando o valor do salário

In [102]:
df['R$SALARIO'] = df['R$SALARIO'].apply(lambda x: float(x.split('R$')[1]))

df['R$SALARIO']

0         1902.24
1         2667.97
2         2302.24
3         1770.00
4         2325.26
           ...   
133907    1100.00
133908    4429.61
133909    2200.00
133910    1635.00
133911    4590.00
Name: R$SALARIO, Length: 133912, dtype: float64

#### Transformar categorias em números

In [103]:
from sklearn.preprocessing import LabelEncoder

df = df.drop(axis=1, columns=['NOME']) # o nome é desnecessário para prever o valor de salário

le = LabelEncoder()

for i in df.columns:
    df[i] = le.fit_transform(df[i])

df

,ORGAO,CATEGORIA,CARGO,R$SALARIO
0,48,4,811,3446
1,48,4,188,6180
2,48,4,811,4954
3,59,3,666,2941
4,48,4,640,5040
...,...,...,...,...
133907,26,3,215,801
133908,26,1,162,11128
133909,26,3,840,4566
133910,26,3,785,2396


### Treinamento do modelo

#### Separando os dados

In [104]:
df_vars_indep = df.drop('R$SALARIO', axis=1)

df_alvo = df[['R$SALARIO']]

from sklearn.model_selection import train_test_split

vars_train, vars_test, res_train, res_test = train_test_split(df_vars_indep, df_alvo, test_size=0.3, random_state=777)

#### Treinando o modelo

In [105]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(vars_train, res_train)

res_prev = model.predict(vars_train) # previsões geradas

res_prev

array([[7579.92744363],
       [4942.30059671],
       [5382.65727362],
       ...,
       [7578.44169842],
       [7578.44169842],
       [5443.70789483]])

#### Medindo performance do modelo

##### Usando dados de treino

In [106]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix

predict_train = model.predict(vars_train)

print(mean_absolute_error(res_train, predict_train)) # medir a média de erro de variável contínua 
print(mean_squared_error(res_train, predict_train)) # grau de intensidade do erro
print(mean_absolute_error(res_train, predict_train) / res_train.mean()) # percent de diferença da média do real com o da previsão

3686.0355633646896
20447558.21201492
R$SALARIO    0.643064
dtype: float64


##### Usando dados de teste

In [107]:
predict_test = model.predict(vars_test)

print(mean_absolute_error(res_test, predict_test))
print(mean_squared_error(res_test, predict_test))
print(mean_absolute_error(res_test, predict_test) / res_test.mean())

3681.0507800815776
20366294.174415722
R$SALARIO    0.642768
dtype: float64


### Conclusão

O modelo consegue previer com 64% de precisão, erra em média cerca de R$3681.05. O modelo poderia se tornar mais preciso se fossem fornecidos dados completos da função de cada cargo